In [1]:
import pandas as pd 
import seaborn as sns
sns.set(font_scale=1)
from plotnine import *
from tqdm.notebook import tqdm
import seaborn as sns
import pandas as pd
from pandarallel import pandarallel

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)
tqdm.pandas()

In [2]:
PAIRED_DATASETS = pd.read_csv("/home/pbarbosa/git_repos/mutsplice/data/2_paired_datasets/ALL_data.tsv.gz", sep="\t")

In [3]:
from gtfhandle.utils import file_to_bed_df
from explainer.datasets.preprocessing import Preprocessing
import os

def runMutSplice(df, rbp_name: str, exon_group: str, motif_source: str, motif_search: str):
    GTF_CACHE = "/home/pbarbosa/data/genomes/hg38/gtf_cache_gencode/"
    OUT_DIR = f"/home/pbarbosa/git_repos/mutsplice/notebooks/2_mutsplice/{motif_source}_{motif_search}/"
    FASTA = "/home/pbarbosa/data/genomes/hg38/GRCh38.primary_assembly.genome.fa"

    kwargs = {
        "gtf_cache": GTF_CACHE,
        "fasta": FASTA,
        "out_dir": f"{OUT_DIR}{rbp_name}_{exon_group}",
        "outbasename": f"{rbp_name}",
        "subset_rbps": [rbp_name],
        "motif_source": motif_source,
        "motif_search": motif_search,
        "pvalue_threshold": 0.00005,
        "spliceai_save_raw": True,
        "spliceai_raw_results": None,
        "spliceai_final_results": None,
        "batch_size": 64,
        "use_full_sequence": False,
    }
    bed_df = file_to_bed_df(df, is_0_based=False, header=0, col_index=0)

    Preprocessing(
        bed_df,
        do_gtf_queries=True,
        do_motif_scanning=True,
        do_mutations=True,
        run_spliceai=True,
        **kwargs,
    )


def process_single_RBP(group, exon_group: str, motif_source: str, motif_search: str):
    try:
        rbp_name = group.iloc[0].rbp_name
        runMutSplice(
            group,
            rbp_name=rbp_name,
            exon_group=exon_group,
            motif_source=motif_source,
            motif_search=motif_search,
        )

    except ValueError as ve:
        print(f"Caught ValueError for {rbp_name}: {ve}")

2024-02-29 12:53:59.920848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 12:54:00.807409: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-29 12:54:00.807488: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-02-29 12:54:00.807493: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

#### Knockdown exons

In [4]:
pandarallel.initialize(progress_bar=True, nb_workers=10)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

KD_data = PAIRED_DATASETS[PAIRED_DATASETS.exon_group == "KD"]
KD_data.groupby("rbp_name").parallel_apply(
    process_single_RBP, exon_group="KD", motif_source="ATtRACT", motif_search="fimo"
)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


2024-02-28|16:11:44 | MAIN: Reading cache file to extract genomics features..


2024-02-28|16:11:52 | MAIN: Scanning motifs..
2024-02-28|16:11:53 | MAIN: No single match found by FIMO.


Caught ValueError for RBFOX2: No motifs found given this experimental setup.


""


#### Control exons 

In [ ]:
pandarallel.initialize(progress_bar=True, nb_workers=10)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Ctrl_data = PAIRED_DATASETS[PAIRED_DATASETS.exon_group == "CTRL"]
Ctrl_data.groupby("rbp_name").parallel_apply(
    process_single_RBP,
    exon_group="CTRL",
    motif_source="ATtRACT",
    motif_search="fimo",
)